In [4]:
from datetime import datetime, timedelta
import sqlite3
import xlsxwriter
import os
import math

In [5]:
def round_to_quarter_hour(minutes):
    return round(minutes / 15) * 15

In [6]:
def convert_minutes_to_hours(minutes):
    return minutes / 60

In [19]:
def weekly_report(
    db: str,
    week_end_date: str,
    employee_group: str
):
    try:
        # Parse the provided date
        week_end = datetime.strptime(week_end_date, "%Y-%m-%d")
        week_start = week_end - timedelta(days=6)

        # Connect to the database
        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        # Fetch the group ID from the employee_group table
        cursor.execute("SELECT id FROM employee_group WHERE name = ?", (employee_group,))
        group_id = cursor.fetchone()
        if not group_id:
            print(f"Employee group '{employee_group}' not found.")
            return
        group_id = group_id[0]

        # Fetch the employee IDs for the selected group
        cursor.execute("SELECT employeeID FROM group_member WHERE groupID = ?", (group_id,))
        employee_ids = [row[0] for row in cursor.fetchall()]

        if not employee_ids:
            print(f"No employees found in group '{employee_group}'.")
            return

        # Prepare SQL query to fetch all punches for the previous week for the employees in the group
        placeholders = ",".join("?" for _ in employee_ids)
        sql_query = f"""
            SELECT 
                e.name,
                pc.id,
                pc.punchInTime,
                pc.punchOutTime,
                pc.punchInApproval,
                pc.punchOutApproval,
                pc.ignoreLunchBreak
            FROM punch_clock pc
            JOIN employee e ON pc.employeeID = e.id
            WHERE pc.punchInTime BETWEEN ? AND ? AND e.id IN ({placeholders})
            ORDER BY e.name, pc.punchInTime
        """

        params = [week_start, week_end] + employee_ids

        # Execute the query
        cursor.execute(sql_query, params)
        punches = cursor.fetchall()

        # Prepare to fetch work punches
        punch_data = {}
        for punch in punches:
            name, punch_id, punch_in, punch_out, in_approval, out_approval, ignore_lunch = punch

            # Fetch work punches for each punch clock entry
            cursor.execute("""
                SELECT 
                    wt.punchType,
                    wt.timeSpent,
                    wt.timeStarted
                FROM work_time wt
                WHERE wt.punchID = ?
                ORDER BY wt.timeStarted
            """, (punch_id,))
            work_punches = cursor.fetchall()

            if name not in punch_data:
                punch_data[name] = []
            punch_data[name].append((punch, work_punches))

        conn.close()

        if not punches:
            print(f"No punches found for the week ending on {week_end_date}.")
            return

        # Create an Excel file
        file_path = f"Weekly_Report_{week_end_date}.xlsx"
        workbook = xlsxwriter.Workbook(file_path)

        for employee, employee_punches in punch_data.items():
            worksheet = workbook.add_worksheet(employee)

            # Write the report headers
            headers = [
                "Date", "Punch In", "Punch Out", "Total Hours Worked", "Construction Hours", 
                "Service Hours", "Office Hours", "Lunch Hours", "Shop Hours"]
            for col, header in enumerate(headers):
                worksheet.write(0, col, header)

            # Initialize a dictionary to store daily work time details
            daily_work_time = {}

            for punch, work_punches in employee_punches:
                _, punch_id, punch_in, punch_out, in_approval, out_approval, ignore_lunch = punch
                punch_in_date = punch_in.split(" ")[0]  # Extract the date part
                punch_out = punch_out if punch_out else datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                if punch_in_date not in daily_work_time:
                    daily_work_time[punch_in_date] = {
                        "total_worked": 0,
                        "construction": 0,
                        "service": 0,
                        "office": 0,
                        "lunch": 0,
                        "shop": 0 }

                punch_in_time = datetime.strptime(punch_in, "%Y-%m-%d %H:%M:%S")
                punch_out_time = datetime.strptime(punch_out, "%Y-%m-%d %H:%M:%S")
                total_minutes = (punch_out_time - punch_in_time).total_seconds() / 60
                rounded_minutes = round_to_quarter_hour(total_minutes)
                daily_work_time[punch_in_date]["total_worked"] += rounded_minutes

                for work_punch in work_punches:
                    punch_type, time_spent, time_started = work_punch
                    time_spent_hours = convert_minutes_to_hours(time_spent)
                    daily_work_time[punch_in_date][punch_type.lower()] += time_spent_hours

                if not ignore_lunch:
                    daily_work_time[punch_in_date]["lunch"] += 0.5

            row = 1
            for work_date, work_details in daily_work_time.items():
                total_worked_hours = convert_minutes_to_hours(work_details["total_worked"])
                shop_hours = total_worked_hours - ( work_details["construction"] + work_details["service"] + work_details["office"] + work_details["lunch"])
                row_data = [work_date, punch_in, punch_out, total_worked_hours, work_details["construction"], work_details["service"], work_details["office"], work_details["lunch"], max(shop_hours, 0)]  # Ensure that shop hours are not negative
                
                for col, data in enumerate(row_data):
                    worksheet.write(row, col, data)
                row += 1

        workbook.close()

        # Send the Excel file to the specified reports channel
        # reports_channel_id = int(os.getenv('TIMECARD_REPORTS_CHANNEL_ID'))
        # reports_channel = self.bot.get_channel(reports_channel_id)
        # if reports_channel:
        #     await reports_channel.send(file=discord.File(file_path))
        print(f"Weekly report for the week ending on {week_end_date} has been generated and sent to the reports channel.")
        # else:
        #     print("Reports channel not found.")
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [22]:
weekly_report(db=f"{os.getcwd()}\\timetracker.db",week_end_date="2024-07-06",employee_group="Hipp Temporary Skills, Inc.")

Weekly report for the week ending on 2024-07-06 has been generated and sent to the reports channel.
